### COPY INTO and MERGE COMMAND

COPY INTO Command
- Incrementally loads data into Delta Lake tables from Cloud Storage
- Supports schema evolution
- Supports wide range of file formats (CSV,JSON, PARQUET,DELTA)
- Alternative to AUTO LOADER for Batch Ingestion

Create the table to copy the data into

In [0]:
%sql
CREATE TABLE IF NOT EXISTS demo.delta_lake.raw_stock_prices;

Incrementally load new files into the table 

In [0]:
%sql
DELETE FROM demo.delta_lake.raw_stock_prices;

COPY INTO demo.delta_lake.raw_stock_prices
from 'abfss://demo@deacourseextdl103.dfs.core.windows.net/stock_prices'
FILEFORMAT = json
FORMAT_OPTIONS ('inferSchema' = 'true')
COPY_OPTIONS("mergeSchema" = "true")

In [0]:
%sql
select * from demo.delta_lake.raw_stock_prices;

### MERGE COMMAND
- used for upserts (insert/update/Delete operations in a single statement)
- Allows merging new data into a target table based on matching condition


CREATE THE TABLE TO MERGE THE DATA INTO 

In [0]:
%sql
CREATE TABLE IF NOT EXISTS demo.delta_lake.stock_prices
(
  stock_id string,
  price DOUBLE,
  trading_date DATE
);

### Merge the source data into target table
- 1.Insert new stock received
- 2.update price and trading_date if updates received
- 3.Delete stocks which are de-listed from the exchange(Status = 'DELISTED')

In [0]:
%sql
select * from demo.delta_lake.stock_prices 

In [0]:
%sql
MERGE INTO demo.delta_lake.stock_prices AS target
using demo.delta_lake.raw_stock_prices AS source
on target.stock_id = source.stock_id
when matched and source.status = 'ACTIVE' THEN 
UPDATE SET target.price = source.price,target.trading_date = source.trading_date
when MATCHED and source.status = 'DELISTED' then DELETE
when not MATCHED and source.status = 'ACTIVE' then 
INSERT (stock_id,price,trading_date)values(source.stock_id,source.price,source.trading_date)


- It has inserted all the records into the table since before we dont have any idea prior

In [0]:
%sql
select * from demo.delta_lake.stock_prices 

now we have addded third file in the cloud, so lets run the merge command again and see the results 

In [0]:
%sql
MERGE INTO demo.delta_lake.stock_prices AS target
using demo.delta_lake.raw_stock_prices AS source
on target.stock_id = source.stock_id
when matched and source.status = 'ACTIVE' THEN 
UPDATE SET target.price = source.price,target.trading_date = source.trading_date
when MATCHED and source.status = 'DELISTED' then DELETE
when not MATCHED and source.status = 'ACTIVE' then 
INSERT (stock_id,price,trading_date)values(source.stock_id,source.price,source.trading_date)

In [0]:
%sql
select * from demo.delta_lake.stock_prices ;